In [561]:
# Import libraries
import requests, ssl
import pandas as pd
import re
import numpy as np
ssl._create_default_https_context = ssl._create_unverified_context
from numpy import nan

In [ ]:
# Download data
dls = "https://www.ect.go.th/ect_th/download/article/article_20201002121316.xlsx"
resp = requests.get(dls, verify = False)

output = open('election62.xlsx', 'wb')
output.write(resp.content)
output.close()

In [562]:
# Create dataframes from excel sheets
num_pol = pd.read_excel("election62.xlsx", header =  3, sheet_name = 0) #จำนวนหน่วยเลือกตั้งในการเลือกตั้งสมาชิกสภาผู้แทนราษฎร
advanced_out = pd.read_excel("election62.xlsx", header =  3, sheet_name = 19) #การใช้สิทธิลงคะแนนเลือกตั้งสมาชิกสภาผู้แทนราษฎรก่อนวันเลือกตั้ง นอกเขตเลือกตั้ง
turnout = pd.read_excel("election62.xlsx", header =  3, sheet_name = 26) #การใช้สิทธิเลือกตั้งสมาชิกสภาผู้แทนราษฎร แบบแบ่งเขตเลือกตั้ง
results = pd.read_excel("election62.xlsx", header =  3, sheet_name = 30) #ผลคะแนนการเลือกตั้งสมาชิกสภาผู้แทนราษฎร แบบแบ่งเขตเลือกตั้ง เมื่อวันที่ 24 มีนาคม 2562
chiangmai_8 = pd.read_excel("election62.xlsx", header =  8, sheet_name = 31) #ผลคะแนนการเลือกตั้งสมาชิกสภาผู้แทนราษฎร กรณีเลือกตั้งใหม่ จังหวัดเชียงใหม่ เขต 8
winners = pd.read_excel("election62.xlsx", header =  3, sheet_name = 37) #รายชื่อผู้ได้รับเลือกตั้งสมาชิกสภาผู้แทนราษฎร แบบแบ่งเขตเลือกตั้ง (จำนวน 350 คน)

In [563]:
# Create a function to clean data
def fill_prov(df):
    df['จังหวัด'] = df['จังหวัด'].fillna(method='ffill')
    df['เขต'] = df['เขต'].fillna(method='ffill')
    df = df[~df['จังหวัด'].str.contains('รวม', na=False)].reset_index(drop=True)
    return df

In [564]:
# Apply the above function to the spreadsheets
df_list = [num_pol, advanced_out, turnout, results, winners]
df_list = [(lambda x: fill_prov(x))(df) for df in df_list]

In [565]:
#จำนวนหน่วยเลือกตั้งในการเลือกตั้งสมาชิกสภาผู้แทนราษฎร
num_pol = df_list[0][['จังหวัด', 'เขต', 'Unnamed: 4']]
num_pol = num_pol.rename(columns={"Unnamed: 4": "จำนวนหน่วยเลือกตั้ง"})

#การใช้สิทธิลงคะแนนเลือกตั้งสมาชิกสภาผู้แทนราษฎรก่อนวันเลือกตั้ง
advanced_out = df_list[1]

#การใช้สิทธิเลือกตั้งสมาชิกสภาผู้แทนราษฎร
turnout = df_list[2]

#ผลคะแนนการเลือกตั้งสมาชิกสภาผู้แทนราษฎร
results = df_list[3].iloc[:-2]

#รายชื่อผู้ได้รับเลือกตั้งสมาชิกสภาผู้แทนราษฎร แบบแบ่งเขตเลือกตั้ง (จำนวน 350 คน)
winners = df_list[4]

#ผลเลือกตั้งซ่อม เชียงใหม่ เขต 8
chiangmai_8 = pd.read_excel("election62.xlsx", sheet_name = 31, header = 8)
chiangmai_8['จังหวัด'], chiangmai_8['เขต'] = 'เชียงใหม่', int(8)
chiangmai_8 = chiangmai_8.dropna(axis=1, how='all')

#การใช้สิทธิเลือกตั้งซ่อม เชียงใหม่ เขต 8
chiangmai_8_turnout = pd.read_excel("election62.xlsx", sheet_name = 31, header = 2)[0:4].iloc[:,2:5]
total_voter = chiangmai_8_turnout.iloc[0, 1]
voter_turnout = chiangmai_8_turnout.iloc[1, 1]
voter_turnout_pct = re.findall('(\d+[,.]?\d*)', chiangmai_8_turnout.iloc[1, 2])[0]
bad_ballot = chiangmai_8_turnout.iloc[2, 1]
bad_ballot_pct = re.findall('(\d+[,.]?\d*)', chiangmai_8_turnout.iloc[2, 2])[0]
vote_no = chiangmai_8_turnout.iloc[3, 1]
vote_no_pct = re.findall('(\d+[,.]?\d*)', chiangmai_8_turnout.iloc[3, 2])[0]

data = [{'จังหวัด': 'เชียงใหม่',
         'เขต': '8', 
         'ผู้มีสิทธิเลือกตั้ง': total_voter,
         'ผู้ใช้สิทธิเลือกตั้ง': voter_turnout, 
         'ร้อยละ': voter_turnout_pct,'บัตรเสีย': bad_ballot, 
         'ร้อยละ.1': bad_ballot_pct,
         'บัตรไม่เลือกผู้สมัครผู้ใด' : vote_no,
         'ร้อยละ.2': vote_no_pct}]
chiangmai_8_turnout = pd.DataFrame(data)
chiangmai_8_turnout['เขต'] = chiangmai_8_turnout['เขต'].astype('int')
chiangmai_8_turnout['index'] = 96
chiangmai_8_turnout = chiangmai_8_turnout.set_index('index')

In [566]:
# Update election turnout with Chiang Mai 8
turnout_cm8_updated = chiangmai_8_turnout.combine_first(turnout)
constituency = pd.merge(turnout_cm8_updated, advanced_out, on = ['จังหวัด', 'เขต']).merge(num_pol, on = ['จังหวัด', 'เขต']).merge(winners, on = ['จังหวัด', 'เขต'])

constituency = constituency.rename(columns={
    "ร้อยละ_x": "ผู้ใช้สิทธิเลือกตั้ง (%)",
    "ร้อยละ.1": "บัตรเสีย (%)",
    "ร้อยละ.2": 'บัตรไม่เลือกผู้สมัครผู้ใด (%)',
    "ผู้ใช้สิทธิลงคะแนน":"ผู้ใช้สิทธิลงคะแนนล่วงหน้านอกเขต",
    "ร้อยละ_y": 'ผู้ใช้สิทธิลงคะแนนล่วงหน้านอกเขต (%)'
})

constituency['ผู้ใช้สิทธิลงคะแนนล่วงหน้านอกเขต (%)'] = (constituency['ผู้ใช้สิทธิลงคะแนนล่วงหน้านอกเขต']/constituency['ผู้ใช้สิทธิเลือกตั้ง'])*100

convert_dict = {'เขต': int,
                'ผู้มีสิทธิเลือกตั้ง': int,
                'ผู้ใช้สิทธิเลือกตั้ง': int,
                'บัตรเสีย': int,
                'บัตรไม่เลือกผู้สมัครผู้ใด': int,
               }
constituency = constituency.astype(convert_dict)

In [531]:
# Update election results with Chiang Mai 8
convert_dict = {'เขต': int,
                'หมายเลข': int,
                'ผลคะแนน': int,
               }
results = results.astype(convert_dict)

subset_index = results.loc[(results['จังหวัด'] == 'เชียงใหม่') & (results['เขต'] == 8)].index.values
chiangmai_8 = chiangmai_8.set_index(subset_index[1:])
results.update(chiangmai_8)
results_updated = results.drop(results.index[subset_index[0]])

In [568]:
# Generate dataframes of top 5 candidates
contenders = {}
df = {}
for i in range(5):
    df[i] = results_updated.groupby(
        ['จังหวัด', 'เขต'], as_index=False).nth(i)[['จังหวัด', 'เขต', 'ชื่อผู้สมัคร', 'พรรค', 'ผลคะแนน']].reset_index(drop= True)
    df[i].rename(columns = {'ชื่อผู้สมัคร':'ชื่อผู้สมัคร ' + str(i+1), 'พรรค':'พรรค ' + str(i+1), 'ผลคะแนน': 'ผลคะแนน ' + str(i+1)}, inplace=True)

In [540]:
# Concatenate into a single dataframe
contenders = pd.concat(df.values(), axis = 1)
contenders = contenders.loc[:,~contenders.columns.duplicated()]

In [547]:
# Update with contenders
out_df = constituency.merge(contenders, on =['จังหวัด', 'เขต'])

In [548]:
# Get margin of victory
out_df['margin of victory (%)'] = ((out_df['ผลคะแนน 1'] - out_df['ผลคะแนน 2'])/ out_df['ผู้ใช้สิทธิเลือกตั้ง'])*100

In [550]:
# Convert select columns to integer
convert_dict = {'ผลคะแนน 1': int,
                'ผลคะแนน 2': int,
                'ผลคะแนน 3': int,
                'ผลคะแนน 4': int,
                'ผลคะแนน 5': int,
               }
out_df = out_df.astype(convert_dict)

convert_dict = {'เขต': int,
                'หมายเลข': int,
                'ผลคะแนน': int,
               }
results_updated = results_updated.astype(convert_dict).reset_index(drop=True)

In [570]:
# Export to .csv 
out_df.to_csv("constituency_62.csv")
results_updated.to_csv("results_62.csv")